# What would be the ideal way to do this?

1. Define points $x_0,\dots,x_{n-1}$ (arbitrary) and pick some $\varepsilon$
2. Define the $\varepsilon$ graph. That is the graph where $i-j$ connect if $D[i,j] \leq \varepsilon$
3. Define the VR complex at $\varepsilon$. Every complete subgraph becomes a simplex. This is where we wonder if enumerating all simplices creates duplicates

Idea: Only generate simplices with increasing vertex order.
$$N^+(i) = \{\, j>i : D[i,j]\leq \varepsilon \,\}$$
I.e. neighbors that come after me.

4. Now we have a simplex $\sigma = [i_0<i_1<\dots<i_k]$. We want to add a new vertex $n$ to make the simplex larger. For $\sigma \cup \{n\}$ to be a simplex, $n$ must connect to every vertex in $\sigma$ in the $\varepsilon$ graph. 

$$n \in N^+(i_0)\cap N^+(i_1)\cap \cdots \cap N^+(i_k)$$

This is why we should maintain a candidate set $C(\sigma) = N^+(i_0)\cap N^+(i_1)\cap \cdots \cap N^+(i_k)$. Now we start at vertex $i$ and find $C([i]) = N^+(i)$. If we extend by adding n, the new candidates become $C(\sigma \cup \{n\}) = C(\sigma)\cap N^+(n)$.

This guarantees that only vertices that connect to all simplex vertices are considered and they respect ordering.

5. If we were to compute all edges, then all triangles, ..., we would store too many simplices. We want to build one simplex at a time, emit its contribution as soon as possible, then backtrack.

# What does Dlotko do compared to this?

1. We already define points in an ascending arbitrary manner with this.
```python
def pairwise_dist(X: np.ndarray) -> np.ndarray: #input is a shape X with n points and d dim
    diff = X[:, None, :] - X[None, :, :] #diff is every pairwise distance vector
    return np.linalg.norm(diff, axis=2)

def subseq_neighbors(D: np.ndarray, i: int, epsilon: float) -> list[int]:
    n = D.shape[0] #precomputed distance matrix D (n is number of points)
    js = np.where((D[i] <= epsilon) & (np.arange(n) > i))[0]
    return js.tolist()
```
subseq_neighbors returns the neighbor set of a vertex with only indices larger than itself ($N^+(i) = \{\, j > i \mid D[i,j] \leq \varepsilon \,\}$). This is how we enforce ordering and avoid duplicate simplices. Let's analyze the line `js = np.where((D[i] <= epsilon) & (np.arange(n) > i))[0]`. `D[i] <= epsilon` is taking the ith row of the distance matrix produces a boolean array containing True is point j is withing $\varepsilon$ of i. `np.arange(n) > 1` creates $[0,1, \dots, n-1]$ and produces a boolean list of True if the indice in greater than i. Now we take the and of these two state ments so we have a boolean list that says True if the point is within $\varepsilon$ distance and its index is $> i$. `np.where` returns the indices where the condition is True, so js is an array of valid neighbors of vertex $i$. 

pairwise_dist takes in a shape $X$ and outputs its symmetric distance matrix $D[i,j] = \|X[i] - X[j]\|$. We can have some issues here with large VR. If $n=10,000$, then $D$ contains 100,000,000 entries. Potentially think about avoiding storing the full distance matrix. 

2. Looking at Algorithm 1

Abstractly what we want to do is:

```python
for each i:
    expand([i], 0, N+(i))
```

We currently have:

```python
for i in range(n):
    simplices = [[i]] #this is a list of a list just like a simplex is a set of a set (this is initializing our simplex)
    filtrations = [0]
    common_subseq_neighs = subseq_neighbors(D, i, epsilon) #this is C = N+(i)
```
These two are identical.


Now we want to `emit(fsigma, (-1)^dim)` and we have:

```python
for sigma, f_sigma in zip(simplices, filtrations):
    dim_sigma = len(sigma) - 1
    sign = 1 if (dim_sigma % 2 == 0) else -1 #this is for EC assigns its sign in the alternating sum
    C.append((f_sigma, sign)) #stores a pair (filtration value, +-1)
```

Now we wish to grow the simplex. 

Ideally:
```python
for n in C:
    f_new = max(fsigma, max_{v in sigma} D[v,n])
    C_new = C intersect N+(n)
```
We have:
```python
for sigma, f_sigma, commonN in zip(simplices, filtrations, common_subseq_neighs):
    for n in commonN:
        sigma2 = sigma + [n]
```


